# Accidentes de tráfico en Reino Unido entre 2010 y 2014 

### Disponible en Kaggle en:
https://www.kaggle.com/stefanoleone992/adm-project-road-accidents-in-uk

### Variables y significado

* Accident_Index: Accident index
* Latitude: Accident latitude
* Longitude: Accident longitude
* Region: Accident region
* Urban_or_Rural_Area: Accident area (rural or urban)
* X1st_Road_Class: Accident road class
* Driver_IMD_Decile: Road IMD Decile
* Speed_limit: Road speed limit
* Road_Type: Road type
* Road_Surface_Conditions: Road surface condition
* Weather: Weather
* High_Wind: High wind
* Lights: Road lights
* Datetime: Accident datetime
* Year: Accident year
* Season: Accident season
* Month_of_Year: Accident month
* Day_of_Month: Accident day of month
* Day_of_Week: Accident day of week
* Hour_of_Day: Accident hour of day
* Number_of_Vehicles: Accident number of vehicles
* Age_of_Driver: Driver age
* Age_of_Vehicle: Vehicle age
* Junction_Detail: Accident junction detail
* Junction_Location: Accident junction location
* X1st_Point_of_Impact: Vehicle first point of impact
* Driver_Journey_Purpose: Driver journey purpose
* Engine_CC: Vehicle engine power (in CC)
* Propulsion_Code: Vehicle propulsion code
* Vehicle_Make: Vehicle brand
* Vehicle_Category: Vehicle brand category
* Vehicle_Manoeuvre: Vehicle manoeuvre when accident happened
* Accident_Severity: Accident severity

In [1]:
from pyspark.sql import SparkSession
#create a SparkSession
spark = (SparkSession
    .builder
    .appName("Example-3_6")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.2")
    .getOrCreate())

### Leerlo tratando de que Spark infiera el tipo de dato de cada columna, y cachearlo.

In [149]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
accidentesDF = spark\
                .read\
                .option("header", "true")\
                .option("inferSchema", "true")\
                .csv("modeloA accidents_uk.csv")
accidentesDF.cache()

DataFrame[Accident_Index: string, Latitude: double, Longitude: double, Region: string, Urban_or_Rural_Area: string, X1st_Road_Class: string, Driver_IMD_Decile: int, Speed_limit: int, Road_Type: string, Road_Surface_Conditions: string, Weather: string, High_Wind: string, Lights: string, Datetime: string, Year: int, Season: int, Month_of_Year: int, Day_of_Month: int, Day_of_Week: int, Hour_of_Day: double, Number_of_Vehicles: int, Age_of_Driver: int, Age_of_Vehicle: int, Junction_Detail: string, Junction_Location: string, X1st_Point_of_Impact: string, Driver_Journey_Purpose: string, Engine_CC: int, Propulsion_Code: string, Vehicle_Make: string, Vehicle_Category: string, Vehicle_Manoeuvre: string, Accident_Severity: string]

### Discretizar la variable **Age_of_Vehicle** utilizando un bucketizer (sin crear un pipeline) en los puntos de corte (0, 2, 5, 10, 15, 20, 35). La discretización debe quedar en una nueva columna de tipo Double llamada **Edad_Vehiculo**.

In [150]:
from pyspark.ml.feature import Bucketizer

# LÍNEAS EVALUABLES, NO RENOMBRAR LAS VARIABLES
cortes =  [0.0, 2.0, 5.0, 10.0, 15.0, 20.0, 35.0]

bucketizer = Bucketizer(splits=cortes, inputCol="Age_of_Vehicle", outputCol="Edad_Vehiculo")
accidentesBucketizedDF = bucketizer.transform(accidentesDF)


### Crear un nuevo DF donde la columna "Age_of_Driver" ha sido reemplazada por otra de tipo string en la que los valores 1 y 2 son "Adolescente", los valores 3 y 4 por "Joven", los valores 5 y 6 por "Adulto", y los demás valores se dejan sin modificar.

In [192]:
from pyspark.sql import functions as F
accidentesAgeDF = accidentesDF.withColumn("Age_of_Driver",
                                          F.when(
                                              (
                                                  (F.col("Age_of_Driver") == 1) 
                                                  | (F.col("Age_of_Driver") == 2)
                                              ),
                                              "Adolescente"
                                          )
                                          .when(
                                              (
                                                  (F.col("Age_of_Driver") == 3)
                                                  | (F.col("Age_of_Driver") == 4)
                                              ),"Joven"
                                          )
                                          .when(
                                              (
                                                 (F.col("Age_of_Driver") == 5)
                                                 | (F.col("Age_of_Driver") == 6) 
                                              ),"Adulto"
                                          )
                                          .otherwise(
                                              F.col("Age_of_Driver")
                                          )
                                         )
accidentesAgeDF.toPandas()   

,Accident_Index,Latitude,Longitude,Region,Urban_or_Rural_Area,X1st_Road_Class,Driver_IMD_Decile,Speed_limit,Road_Type,Road_Surface_Conditions,...,Junction_Detail,Junction_Location,X1st_Point_of_Impact,Driver_Journey_Purpose,Engine_CC,Propulsion_Code,Vehicle_Make,Vehicle_Category,Vehicle_Manoeuvre,Accident_Severity
0,201001BS70015,51.492045,-0.178376,London,Urban,A,2,30,Dual carriageway,Wet or damp,...,Crossroads,Mid Junction - on roundabout or on main road,Front,Other/Not known,1896,Heavy oil,Volkswagen,Taxi,Going ahead,Slight
1,201001BS70034,51.485058,-0.168572,London,Urban,B,8,30,Single carriageway,Dry,...,T or staggered junction,Mid Junction - on roundabout or on main road,Front,Other/Not known,599,Petrol,Honda,Motorcycle,Going ahead,Slight
2,201001BS70040,51.495653,-0.179097,London,Urban,C,7,30,Single carriageway,Dry,...,Not at junction or within 20 metres,Not at or within 20 metres of junction,Front,Other/Not known,1781,Petrol,Audi,Car,Going ahead,Slight
3,201001BS70045,51.494743,-0.195845,London,Urban,A,7,30,Single carriageway,Wet or damp,...,Not at junction or within 20 metres,Not at or within 20 metres of junction,Offside,Other/Not known,649,Petrol,Other,Motorcycle,Moving off,Slight
4,201001BS70060,51.514170,-0.207760,London,Urban,B,5,30,Single carriageway,Wet or damp,...,Crossroads,Mid Junction - on roundabout or on main road,Front,Other/Not known,600,Petrol,Suzuki,Motorcycle,Going ahead,Slight
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251827,2014981106214,54.850068,-4.925632,Scotland,Rural,B,5,60,Single carriageway,Dry,...,Not at junction or within 20 metres,Not at or within 20 metres of junction,Front,Other/Not known,1000,Petrol,BMW,Motorcycle,Going ahead,Fatal_Serious
251828,2014982105114,55.108757,-4.149116,Scotland,Rural,A,8,60,Single carriageway,Dry,...,Not at junction or within 20 metres,Not at or within 20 metres of junction,Front,Other/Not known,980,Petrol,BMW,Motorcycle,Going ahead,Fatal_Serious
251829,2014982109714,55.158556,-4.195310,Scotland,Rural,A,4,60,Single carriageway,Dry,...,Not at junction or within 20 metres,Not at or within 20 metres of junction,Front,Other/Not known,1100,Petrol,BMW,Motorcycle,Going ahead,Fatal_Serious
251830,2014982109714,55.158556,-4.195310,Scotland,Rural,A,2,60,Single carriageway,Dry,...,Not at junction or within 20 metres,Not at or within 20 metres of junction,Front,Journey as part of work,2143,Heavy oil,Mercedes,Van,Going ahead,Fatal_Serious


###  Partiendo de accidentesDF, crear un nuevo DataFrame de una sola fila que contenga, por este orden de columnas, el número de categorías existentes para el propósito del viaje, para el tipo de maniobra del vehículo, para las condiciones de la calzada y para la severidad del accidente. Pista: crear las columnas al vuelo con select(). Renombrar cada columna de conteo para que se llame igual que la propia columna que estamos contando.

In [152]:
numeroCategoriasDF = accidentesDF.select(
    F.countDistinct("Driver_Journey_Purpose").alias("Proposito_viaje"),
    F.countDistinct("Vehicle_Manoeuvre").alias("Tipo_maniobra"),
    F.countDistinct("Road_Surface_Conditions").alias("Condiciones_calzada"),
    F.countDistinct("Accident_Severity").alias("Severidad_accidente"),
)

In [153]:
numeroCategoriasDF.toPandas()

,Proposito_viaje,Tipo_maniobra,Condiciones_calzada,Severidad_accidente
0,5,11,5,2


### Partiendo de accidentesAgeDF definido anteriormente, crear un nuevo DataFrame con tantas filas como posibles propósitos de un viaje, y tantas columnas como rangos de edad habíamos distinguido en dicho DataFrame más una (la del propósito del viaje). Las columnas deben llamarse igual que cada uno de los niveles posibles de rangos de edad. Cada casilla del nuevo DataFrame deberá contener el porcentaje del número de accidentes ocurridos en ese tipo de viaje (fila) para ese rango de edad (columna), medido sobre el total de accidentes ocurridos para ese tipo de viaje.

In [154]:
viajesPorEdadDF = accidentesAgeDF.groupby("Driver_Journey_Purpose")\
                                 .pivot("Age_of_Driver")\
                                 .count()\
                                 .withColumn("Total",
                                             (F.col("7") + 
                                              F.col("8") + 
                                              F.col("Adolescente") + 
                                              F.col("Joven") + 
                                              F.col("Adulto")
                                             )
                                            )\
                                 .withColumn("7",
                                             F.col("7") / F.col("Total")
                                            )\
                                 .withColumn("8",
                                             F.col("8") / F.col("Total")
                                            )\
                                 .withColumn("Adolescente",
                                             F.col("Adolescente") / F.col("Total")
                                            )\
                                 .withColumn("Joven",
                                             F.col("Joven") / F.col("Total")
                                            )\
                                 .withColumn("Adulto",
                                             F.col("Adulto") / F.col("Total")
                                            )\
                                 .drop("Total")

In [155]:
viajesPorEdadDF.toPandas()

,Driver_Journey_Purpose,7,8,Adolescente,Adulto,Joven
0,Taking pupil to/from school,0.026578,0.009061,0.075506,0.222591,0.666264
1,Pupil riding to/from school,0.021611,0.017682,0.622790,0.125737,0.212181
2,Journey as part of work,0.023331,0.003815,0.155361,0.318026,0.499468
3,Other/Not known,0.065032,0.048146,0.240268,0.256509,0.390045
4,Commuting to/from work,0.012528,0.002520,0.236328,0.279199,0.469425


### Unir la información obtenida en el paso anterior al DataFrame accidentesAgeDF, de manera que al resultado final se añada una columna nueva llamada Porcentaje que contenga el porcentaje de accidentes que ha habido para ese rango de edad y ese tipo de viaje de entre todos los viajes que ha habido de ese tipo (es decir, el porcentaje adecuado de la tabla anterior). Por ejemplo, si el accidente se produjo en un trayecto de tipo Commuting... y la persona es Joven, entonces la columna Porcentaje tomará el valor de la columna Joven y por tanto tendrá el valor 0.46942, pero si la persona es Adulto, entonces tomará el valor de la columna Adulto el cual será 0.2791993469851297.

In [202]:
finalDF = accidentesAgeDF.join(viajesPorEdadDF, 
                               "Driver_Journey_Purpose"
                              ).withColumn("Porcentaje", 
                                           F.when(F.col("Age_of_Driver") == "Joven", F.col("Joven"))\
                                            .when(F.col("Age_of_Driver") == "Adulto", F.col("Adulto"))\
                                            .when(F.col("Age_of_Driver") == "Adolescente", F.col("Adolescente"))\
                                            .when(F.col("Age_of_Driver") == "7", F.col("7"))\
                                            .when(F.col("Age_of_Driver") == "8", F.col("8"))\
                                            .otherwise(None) 
                                          )
                               
finalDF.select("Driver_Journey_Purpose", "Age_of_Driver", "Porcentaje").toPandas()

,Driver_Journey_Purpose,Age_of_Driver,Porcentaje
0,Taking pupil to/from school,Adolescente,0.075506
1,Taking pupil to/from school,Joven,0.666264
2,Taking pupil to/from school,Adulto,0.222591
3,Taking pupil to/from school,Adulto,0.222591
4,Taking pupil to/from school,Joven,0.666264
...,...,...,...
251827,Commuting to/from work,Joven,0.469425
251828,Commuting to/from work,Joven,0.469425
251829,Commuting to/from work,Joven,0.469425
251830,Commuting to/from work,Joven,0.469425


In [209]:
finalDF = finalDF.select("Driver_Journey_Purpose", "Age_of_Driver", "Porcentaje")
finalDF.groupby("Driver_Journey_Purpose","Age_of_Driver")\
       .avg()\
       .withColumn("Porcentaje",F.col("avg(Porcentaje)"))\
       .drop("avg(Porcentaje)")\
       .toPandas()

#select("Driver_Journey_Purpose", "Age_of_Driver", "Porcentaje").toPandas()

,Driver_Journey_Purpose,Age_of_Driver,Porcentaje
0,Taking pupil to/from school,Adolescente,0.075506
1,Taking pupil to/from school,Joven,0.666264
2,Taking pupil to/from school,Adulto,0.222591
3,Taking pupil to/from school,7,0.026578
4,Taking pupil to/from school,8,0.009061
5,Pupil riding to/from school,Adolescente,0.622790
6,Pupil riding to/from school,Joven,0.212181
7,Pupil riding to/from school,8,0.017682
8,Pupil riding to/from school,Adulto,0.125737
9,Pupil riding to/from school,7,0.021611
